Université Paul Sabatier

EIMAB3H1 - Analyse et exploitation de données

Enseignant : **José G. Moreno**

## TP 1. Extraction d'information sur documents textuels (1/3)
Identifier correctement la présence de noms ou informations dans un texte est une tâche important de
l'extraction d'information. Par exemple dans le texte

```
« Le Front Fatah Al-Cham, qui prétend avoir rompu avec Al-Qaida, mais que 
Washington considère toujours comme terroriste, occupe une place centrale sur
l’échiquier rebelle. D’où la difficulté à départager bons et mauvais rebelles. En
savoir plus sur http://www.lemonde.fr/#ltXX2jh9UwX7vbDM.99 »
```
, les noms ou informations à identifier sont :
- Front Fatah Al-Cham -> Organisation
- Al-Qaida -> Organisation
- Washington -> Lieu
- http://www.lemonde.fr/#ltXX2jh9UwX7vbDM.99 -> URL

ou dans le texte
```
« Un futur pont sur la Saône sera baptisé du nom de Jacques Chirac « en hommage
à l’attachement de l’ancien président de la République aux territoires et à leur unité
», ont annoncé mardi les départements de l’Ain et de la Saône-et-Loire.
Disparu jeudi, « Jacques Chirac était de ceux qui savaient écouter, comprendre et
agir pour le bien de ses concitoyens », soulignent Jean Deguerry, président du
département de l’Ain, et André Accary, président du département de Saône-et-Loire,
dans un communiqué commun ».
```

- Saône -> Lieu
- Jacques Chirac -> Personne
- Ain -> Lieu
- Saône-et-Loire -> Lieu
- Jean Deguerry -> Personne
- André Accary -> Personne

Dans ces deux exemples, nous nous limitons à quatre types de noms ou informations (Organisation,
Lieu, Personne et URL), cependant des autres informations peuvent nous intéresser comme les dates,
les adresses mails, les noms de produits, les événements, etc, dans la littérature, ces noms ou
informations sont appelés des « entités nommées ». 

### I - L'usage des dictionnaires
Pour identifier certains types d’entités nommées l'utilisation de dictionnaires est courante. Par exemple,
l'identification d'un lieu comme un ville ou un pays. Nous allons construire un dictionnaire de pays à
partir de Wikipédia simple (https://simple.wikipedia.org/wiki/Main_Page). 


Wikipédia est une des ressources disponibles sur Internet le plus complètes et de
libre accès. Elle est aussi très utilisé comme source d'information par plusieurs entreprises. Par exemple
le projet Freebase de Google a été créé à partir de donnés extrait de Wikipédia parmi autres sources.
Aujourd'hui devenu le Knowledge Graph est la base de connaissance utilisé par Google pour améliorer
son moteur de recherche.
Parmi toutes les informations disponibles sur Wikipédia, la listes de listes est une source important
d'information (pour la Wikipédia en anglais utilisez l'article List of list of list). Avec l'article respective,
nous allons générer un dictionnaire de la liste des pays par exemple :


In [ ]:
!wget https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-pages-articles.xml.bz2
!bzip2 -d simplewiki-20200901-pages-articles.xml.bz2
! echo "<mediawiki>" > simplewiki-20200901-pages-articles.xml.clean
! tail -n +2 simplewiki-20200901-pages-articles.xml >> simplewiki-20200901-pages-articles.xml.clean

--2020-11-08 18:28:18--  https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179777498 (171M) [application/octet-stream]
Saving to: ‘simplewiki-20200901-pages-articles.xml.bz2’

simplewiki-20200901 100%[===================>] 171.45M  4.68MB/s    in 36s     

2020-11-08 18:28:54 (4.79 MB/s) - ‘simplewiki-20200901-pages-articles.xml.bz2’ saved [179777498/179777498]



In [ ]:
import pandas as pd
import xml.etree.ElementTree as et

def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
      if node.find(df_cols[0]) is not None:
        res = [node.find(df_cols[0]).text]
        for el in df_cols[1:]: 
            if node is not None and node.find('revision',).find(el,) is not None:
                res.append(node.find('revision',).find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [ ]:
df = parse_XML("simplewiki-20200901-pages-articles.xml.clean", ["title", "text"])
df

,title,text
0,April,{{monththisyear|4}}\n'''April''' is the fourth...
1,August,{{monththisyear|8}}\n'''August''' (Aug.) is th...
2,Art,[[Category:Art| ]]\n[[Category:Non-verbal comm...
3,A,{{about| the first [[letter]] in the [[alphabe...
4,Air,{{Simplify|date=February 2020}}\n[[Image:Kawas...
...,...,...
319768,Workers and Peasants Party,Bhagat sing was the grastest Second freedom fi...
319769,Vladislav Krapivin,[[File:Krapivin 2006.jpg|right|200px]]\n'''Vla...
319770,Tomomi Inada,{{Infobox officeholder\n|name = Tomo...
319771,Shigeru Ishiba,{{Infobox officeholder\n|name = Shig...


Nous allons accéder à la page  Wikipédia « List of countries » qui contient la liste des pays :

In [ ]:
print(df[df['title']=='List of countries']['text'].values)

['This is a list of [[sovereign state]]s.\n\n{| class="sortable wikitable" style="background:white; text-align:left;"\n|-\n! style="width:35%" |Common and formal names\n! style="width:12.5%;" |Membership within the [[United Nations System|UN System]]{{efn|This column indicates whether or not a state is a member of the [[United Nations]]. It also indicates which non-member states participate in the [[United Nations System]] through membership in the [[International Atomic Energy Agency]] or one of the [[List of specialized agencies of the United Nations|specialized agencies of the United Nations]]. All United Nations members belong to at least one specialized agency and are parties to the statute of the [[International Court of Justice]].}}\n! style="width:12.5%;" |Sovereignty dispute{{efn|This column indicates whether or not a state is the subject of a major sovereignty dispute. Only states whose entire sovereignty is disputed by another state are listed.}}\n! class="unsortable" |Furth

Voici la liste final dans une variable de type [set](https://docs.python.org/2/library/sets.html)

In [ ]:
import re

s = str(df[df['title']=='List of countries']['text'].values)
pattern = re.compile(r"\[\[([A-Za-z0-9_ ]+)\]\]")

my_countries = set([x.group(0)[2:-2] for x in pattern.finditer(s)])
my_countries

Utilisez le code ci-dessus pour accèder à autres pages et extraire des dictionnaires pour les villes, présidents, personnages fictifs, etc.

In [ ]:
s = str(df[df['title']=='Lists of communes of France']['text'].values)
my_communes = set([x for x in pattern.findall(s)])
my_communes

# pour les villes : Lists of communes of France
# pour les présidents : List of presidents of the United States
# pour les personnages fictifs : Fictional character
# etc...

### II - L'usage des règles
Les listes ou dictionnaires sont utiles pour identifier les entités nommées, mais ce n'est pas le cas dans
la plus part de documents. Dans le deuxième exemple, la chaîne de caractères André Accary est
identifié comme un entité nommée de type personne, mais ce nom n’ai pas partie d'un dictionnaire des
personnes extrait depuis la Wikipédia (car la page n’existe pas). L'usage de règles est une alternative
viable dans ce cas. Par exemple, la règle « mort de TOKEN » (ou sont équivalent en anglais « death
of ») peut être utile pour identifier que TOKEN est une entité nommée de type Personne. TOKEN
peut être aussi défini comme deux mots qui commencent par une majuscule. Voici le code pour
appliquer cette règle sur Wikipédia :

In [ ]:
# def findOrga(text): 
#     pattern1 = "organization of "
#     pattern =  " organization"
#     return(text)
#     # for val in re.finditer(' organization of [A-Z]\w+( [A-Z]\w+)*', text):
#     #   # print (val.group(0))
#     #   # print(text)
#     #   liste = list()
#     # for val in re.finditer('[A-Z]\w+( [A-Z]\w+)*'+pattern, text):
#     #   # print(val.group(0))

# for page in df[1:10]['text'] :
#   print(findOrga(page))

In [ ]:
def findPeople(s): 
  pattern = re.compile(r"death of ([A-Z]\w+ [A-Z]\w+)")
  for x in pattern.finditer(str(s)):
    print(str(x.group(0)[9:]))

for page in list(df['text']):
  findPeople(page)

In [ ]:
#modification du code du dessus pour enlever les doublons
def findPeople(s, uneListe): 
    pattern = re.compile("death of ([A-Z]\w+ [A-Z]\w+)")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

listePeople = list()
for page in list(df['text']):
    findPeople(page,listePeople)
print(set(listePeople))


{'Gavin Watson', 'Pulikeshin II', 'Dr Lo', 'Janine Sutto', 'Kurt Cobain', 'John Paul', 'Cheo Feliciano', 'Charles Jastrow', 'George Francis', 'Raymonde Linossier', 'Sub Officer', 'Emperor Taizu', 'General Anson', 'Victor Amadeus', 'Imam Hussain', 'Italian World', 'Goa Chief', 'British World', 'Lee Batchelor', 'Tipu Sultan', 'Henry Surtees', 'Darth Vader', 'Emperor Chongzhen', 'King Saul', 'Barron Hilton', 'Francis Winslow', 'Mario Jeckle', 'Tom Alexander', 'Baldwin II', 'John Cabot', 'Qu Yuan', 'Father Hidalgo', 'Prince Edward', 'Henry Dashwood', 'Francis II', 'Eloy Inos', 'Tom Mitchell', 'Samuel Huntington', 'Regis Korchinski', 'Wendell Bell', 'Margaret Mary', 'Rachel Donelson', 'George Floyd', 'Louis XV', 'Father François', 'Charles XII', 'Fred Hampton', 'Nasiruddin Mahmood', 'Michael Jackson', 'Wild Bill', 'Charles Kingston', 'Jackie Johns', 'Empress Wu', 'Roberta Cowell', 'Alberta King', 'Edmund Perry', 'General Barnard', 'Murtaja Baseer', 'Johann Wilhelm', 'John Price', 'Lionel Re

1) Définir et implémenter des règles pour identifier des entités nommées type Pays et comparer avec la
liste qui nous avons extrait auparavant (sans utiliser la page « List of countries »). Quelle pays ont été
simples à extraire et pourquoi ? Avez-vous réussir à extraire des pays qui ne font pas partie de la liste ?
Pourquoi ? Comment évaluer la pertinence d'une règle ?


In [ ]:
#Nouvelle dataframe sans la page "List of countries"
nouvDF = df.drop(df.loc[df['title']=='List of organisations'].index)

#recherche de pays sur chacune des pages restantes
def findCountry(s, uneListe): 
    #pattern = re.compile('born in ([A-Z]\w+( [A-Z]\w+)*)')
    pattern = re.compile("live in \[\[([A-Za-z_ ]+)\]\]")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

listeCountry = list()
for page in list(nouvDF['text']):
    findCountry(page,listeCountry)
print(set(listeCountry))




{'city park', 'Taiwan', 'human', 'Basel', 'South America', 'Chigwell', 'Tahiti', 'desert', 'rainforests', 'brook', 'Cherdyn', 'Aleppo', 'Florida', 'Yingxiu', 'Asgard', 'Sudan', 'Latin America', 'Prague', 'Ludlow Castle', 'Great Britain', 'New York City', 'town', 'Finland', 'arid', 'Mindanao', 'Jammu and Kashmir', 'savanna', 'Hertfordshire', 'Parma', 'castle', 'society', 'Mongolia', 'cloud forest', 'Alaska', 'saltwater', 'hydrothermal vents', 'mainland', 'Tibet', 'savannas', 'Algeria', 'Eastern Arc', 'Christianity', 'caves', 'rural', 'British Columbia', 'Massachusetts', 'cities', 'freshwater', 'Belize', 'Leiden', 'the Americas', 'England', 'rainforest', 'Loudun', 'Lake Saimaa', 'Japan', 'Bolivia', 'Gambia', 'Vienna', 'grassland', 'Malawi', 'herd', 'Madagascar', 'tepee', 'Brooklyn', 'rotting', 'Bilbao', 'Pakistan', 'Copenhagen', 'North Africa', 'Lille', 'root', 'Milano', 'Guyana', 'Azerbaijan', 'wetland', 'Singapore', 'Italy', 'dens', 'Stavanger', 'Toronto', 'Siberia', 'salt water', 'Mau

les pays composés d'un seul mot et commençant par une majuscule sont les plus simples à extraire

In [ ]:
for ville in listeCountry :
    if (ville not in my_countries):
        print(ville)

En lançant se code là on trouve des pays non présents dans la liste
par exemple la France, Iran n'apparaissent pas dans votre liste...

On a réussi à éxtraire ces pays différents car ce sont des pages différentes qui traitent de thématiques différentes donc des pays abordés différents

On peut évaluer la pertinence d'une règle en calculant la précision et le rappel

2) Définir et implémenter des règles pour identifier des lieux, des organisations, des personnes (morts
et vivants), etc.


In [ ]:
def findLieu1(s, uneListe): 
    pattern = re.compile("from \[\[([A-Za-z_ ]+)\]\]")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

def findOrganisation1(s, uneListe): 
    pattern = re.compile("member of \[\[([A-Za-z_ ]+)\]\]")
    #pattern = re.compile("the \[\[([A-Za-z_ ]+)\]\] was founded")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

def findPersonne1(s, uneListe): 
    pattern = re.compile("mother of \[\[([A-Za-z_ ]+)\]\]")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

listeLieu = list()
listeOrganisation = list()
listePersonne = list()
for page in list(nouvDF['text']):
  # findLieu1(page,listeLieu)
  # findOrganisation1(page,listeOrganisation)
    findPersonne1(page,listePersonne)

print(set(listePersonne))
# print(set(listeOrganisation))
# print(set(listeLieu))



{'Stephen Lawrence', 'Kajol', 'John McCain', 'Queen Elizabeth The Queen Mother', 'Saint Casimir', 'Selena', 'Yeonsangun of Joseon', 'Romulus and Remus', 'Winston Churchill', 'Mitt Romney', 'John Lennon', 'Julius Caesar', 'Matilda of Flanders', 'Centeotl', 'Jennifer Aniston', 'Minotaur', 'Nero', 'Athena', 'Lady Jane Grey', 'Louis XV of France', 'Louis XVII of France', 'Wilhelmina of the Netherlands', 'Bill Clinton', 'Abraham Lincoln', 'Angelina Jolie', 'Bob Marley', 'Marie Antoinette', 'Louis XV', 'Piper McLean', 'Wolfgang Amadeus Mozart', 'Hermes', 'Walt Disney', 'Elizabeth of York', 'Donald Trump', 'Rupert Murdoch', 'Odin', 'Queen Victoria', 'Tupac Shakur', 'Pan', 'Princess Hetepheres', 'Chris Blackwell', 'Henry VIII', 'Thin Lizzy', 'Arkas', 'Herakles', 'Zelus', 'Gungsong Gungtsen', 'Napoleon III of France', 'Queen Juliana', 'John Ritter', 'Shirley Temple', 'Ryan Reynolds', 'Jason Ritter', 'Solymos', 'Shirley Williams', 'Astyanax', 'Hillary Clinton', 'Selene', 'Brett Whiteley', 'Thala

3) Définir au moins 3 règles dans lesquelles l'entité nommée n'est pas dans une extrémité de la chaîne
de caractères.


In [ ]:
def findLieu2(s, uneListe): 
    pattern = re.compile("the \[\[([A-Za-z_ ]+)\]\] located ")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

def findOrganisation2(s, uneListe): 
    pattern = re.compile("the \[\[([A-Za-z_ ]+)\]\] was founded")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

def findPersonne2(s, uneListe): 
    pattern = re.compile("where \[\[([A-Za-z_ ]+)\]\] was")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

listeLieu2 = list()
listeOrganisation2 = list()
listePersonne2 = list()
for page in list(nouvDF['text']):
    findLieu2(page,listeLieu2)
    # findOrganisation2(page,listeOrganisation2)
    findPersonne2(page,listePersonne2)

print(set(listePersonne2))
# print(set(listeOrganisation2))
# print(set(listeLieu2))

{'Moses', 'iron', 'Serbia', 'Ned Kelly', 'Medusa', 'capitalism', 'Nelson Mandela', 'Adam', 'Felix Benda', 'coffee', 'George Washington', 'Garmin', 'photography', 'golf', 'Bhagavad Gita', 'Simon Bolivar', 'gunpowder', 'General Motors', 'radiocarbon dating', 'iron ore', 'Hubert Parry', 'Scott Morrison', 'Thomas Becket', 'Abraham Lincoln', 'Detective Van Zwam', 'Emmett Till', 'Jesus', 'paper', 'north', 'Billy Graham', 'Pablo Picasso', 'Checkpoint Charlie', 'Chris Jericho', 'FM radio'}


4) Définir au moins 3 règles qu'utilise la ponctuation pour trouver les entités nommées.


In [ ]:
def findPays(s, uneListe): 
    pattern = re.compile("\[\[([A-Za-z_ ]+)\]\] ;")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

def findPersonne3(s, uneListe): 
    pattern = re.compile("\- \[\[([A-Za-z_ ]+)\]\] ?")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))

def findLieu3(s, uneListe): 
    pattern = re.compile("\[\[([A-Za-z_ ]+)\]\] =")
    for x in pattern.findall(str(s)):
        uneListe.append(str(x))


listePays = list()
listeLieu3 = list()
listePersonne3 = list()
for page in list(nouvDF['text']):
  # findLieu3(page,listeLieu3)
  # findPersonne3(page,listePersonne3)
    findPays(page,listePays)

# print(set(listePersonne3))
# print(set(listeLieu3))
print(set(listePays))

{'Freddie Perren', 'Seychelles', 'International Cricket Council', 'France', 'Belgium', 'software engineering', 'Main Page', 'Brassica', 'Giorgio Moroder', 'ESSEC', 'New Caledonia', 'romanticism', 'Thunder Bay', 'energy', 'Ruth Pointer', 'ISIPCA', 'Yvette Guilbert', 'Bette Midler', 'CIA', 'Foraminifera', 'Kozhikode', 'amoeba'}


5) Choisissez aléatoirement 10 pages Wikipédia et appliquez toutes les règles qui vous avez défini.
Imprimez les documents avec les entités nommées identifiées. Il y a des manquants ? Il y a des entités
nommées de trop ? Évaluez la performance des vos règles en calculant la [precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)  et le [rappel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score) de
votre implémentation pour ces 10 documents

In [ ]:
# Sur 10 pages aléatoires je ne trouve presque rien je n'ai pas pu calculer la précision et
# le rappel (si je le calculais ça faisait presque toujours 0)

def recherche10page(df,n):
    #Extraire 10 textes au hasard
    df_extrait = df.sample(n=10, random_state=2)

    #Création de toutes les listes
    listeLieu1,listeLieu2,listeLieu3 = list(),list(),list()
    listeOrganisation1,listeOrganisation2 = list(),list()
    listePersonne1,listePersonne2,listePersonne3 = list(),list(),list()

  #application de toutes les fonctions
      for texte in df_extrait['text']:
        findLieu1(texte,listeLieu1),findLieu2(texte,listeLieu2),findPays(texte,listeLieu3)
        findOrganisation1(texte,listeOrganisation1),findOrganisation2(texte,listeOrganisation2)
        findPersonne1(texte,listePersonne1),findPersonne2(texte,listePersonne2),findPersonne3(texte,listePersonne3)

    return(listeLieu3,listeOrganisation1,listePersonne1)

Pays,Organisation,Personne = recherche10page(df,10)
print(Pays)

# Les formules qu'il aurait fallu utiliser si on avait des sorties

#précision = entités trouvées vraies / (entités trouvées vraies + entités trouvées fausses)
#rappel = entités trouvées vraies / (entités qu'il fallait trouver dans le texte)

[]
